# Dimension Reduction 예제

## 신용카드를 소지한 고객 데이터: 17개 변수로 구성 
예: 구매액, 현금서비스, 사용한도금액, 최소결제금액, 완납비율, 보유기간 등

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("creditcard.csv")
df = df.drop('CUST_ID',axis=1)

#### 결측치 확인 및 처리

In [ ]:
df = df.fillna(df.median())
df.isnull().sum().sum()

#### 우측 치우침이 심한 변수가 많음
#### 로그 변환을 수행 (0에는 로그를 취할 수 없으므로, 1을 더한 후 로그변환)

In [ ]:
xvar = df.columns
df1 = df.copy()
xvarlog = ['BALANCE', 'PURCHASES','ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',     
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS','MINIMUM_PAYMENTS']
df1[xvarlog] = np.log(1+df1[xvarlog])

## 차원축소
#### 상관관계 강한 변수들 확인하여 차원축소 필요성 점검

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df1.corr(),cmap='coolwarm',annot=True)
plt.show()

#### 주성분 분석, Scree Plot

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
df1_std = scaler.fit_transform(df1)
df1_std = pd.DataFrame(df1_std)
df1_std.columns = xvar

In [ ]:
pca = PCA() 
pca.fit(df1_std) 
pca.explained_variance_  # eigenvalues

In [ ]:
plt.plot(pca.explained_variance_, 'o-')
plt.show()

In [ ]:
np.cumsum(pca.explained_variance_) / len(xvar)# eigenvalue 누적 비율

#### 주성분 2개 추출

In [ ]:
pca2 = PCA(n_components=2) 
pca2.fit(df1_std) 
pca2.explained_variance_  # eigenvalues

In [ ]:
pca2.components_  # eigenvectors

In [ ]:
df1_pc2 = pca2.fit_transform(df1_std) 
df1_pc2 = pd.DataFrame(df1_pc2) 
df1_pc2.columns = ['PC1', 'PC2'] 
df1_pc2

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(data=df1_pc2, x="PC1", y="PC2", alpha = 0.05)
plt.show()

In [ ]:
sns.displot(data=df1_pc2, x="PC1", y="PC2", kind="kde")
plt.show()

#### t-SNE 방법

In [ ]:
from sklearn.manifold import TSNE
np.random.seed(0)
tsne = TSNE(n_components=2, perplexity=30) # perlexity 는 이웃의 범위
df1_tsne2 = tsne.fit_transform(df1_std)

In [ ]:
df1_tsne2 = pd.DataFrame(df1_tsne2) 
df1_tsne2.columns = ['tsne1', 'tsne2'] 
df1_tsne2

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(data=df1_tsne2, x="tsne1", y="tsne2", alpha = 0.05)
plt.show()

In [ ]:
sns.displot(data=df1_tsne2, x="tsne1", y="tsne2", kind="kde")
plt.show()